In [ ]:
# Further analysis of decoupled questions

**Questions to be answered:**
- 4.1: Why less electrolysis capacity in EXP? shouldn't this have more as the strategy is wind > hydrogen and it has more wind?)
- 4.2: What is interesting is, that electrolysis which is an adjustable technology is also still producing at these high prices. (special case), Why is electrolysis producing the whole year?
- 4.2: Why are the capacity factors exactly the same for some regions (hydro in Norway?)
- 4.2: How is it that V2G has only so low generation when they discharge by 0.2 and have a max capacity of 200 times the one as the battery??? Is this plot over-representing V2G? Fig. 4.19
- 4.2: Reasons why some regions do not need peak plants at all: Iberian Peninsula: warm winters and therefore no need for CHP plants, British islands: enough wind power? but they have OCGT, scandinavia: enough hydro power?}
- 4.3: This effect is also present in the correlation of market values and generation which is -0.37 in the STST case and -0.08 in the EXP case. More transmission reduces the locality of market values and reduces the impact of the location of the generation on the market values. This leads to a smaller range and therefore a more evenly distribution.
- 5.1: Why no SMR CC in EXP? Why much more Sabatier in STST?} Weil stst methan aus sabatier im gas netz transportiert und somit die netzrestriktionen reduziert. Es kommt vor, dass methan produziert wird dann woanders hingepiped und dann wieder verbrannt wird. -> zeige, dass methane in GB produziert und woanders genutzt wird? Also ins Gasnetz eingespeist wird.
- 5.1: (How is it possible that electrolysis is temporally negative correlated with PHS and PHS (con)? )
- 5.2: Capacity factors: Fig. 5.8. Why are the values for Sweden and Norway so high in STST? Why is the generation weighted capacity factor in STST and EXP the same?? How is teh difference in capacity factors in the major producing regions? Maybe the capacity factors are less for EXP there because the capacity is higher there?
- 5.2: (What is all the additional methane in STST used for?)
- 5.2: Relationship of capacity factors and VRE generation: Frischmuth adn Härtel find a positive correlation. I find a negative correlation. Does that make sense?
- 5.2: Make hydrogen storage flow analysis again!
- 5.3: Distribution of congestion rent? Reasons behind?

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import math
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from datetime import date, datetime, time, timedelta

# imported own functions
from utils import generation_links_bus, congestion_rent_link

# imported own definitions
from utils import carrier_colors, carrier_renaming, carrier_renaming_reverse, c_tags
from utils import resistive_heater, gas_boiler, heat_pump,water_tanks_charger, water_tanks_discharger, solar_thermal
from utils import c_el_gen_s, c_el_con_s, c_h2_gen, c_h2_con
from utils import convert_ISO_3166_2_to_1

# general variables
font1 = {'fontname': 'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/5.3_hydrogen_mv'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]
markers = ["v", "^", "<", ">", "1", "2", "3", "4", "*", "+", "d", "o", "|", "s", "P", "p", "h"]

# new indices: cut off Fuel cell and SMR completely and SMR CC for exp
c_h2_gen_stst = ['H2 Electrolysis', 'SMR CC']
c_h2_gen_exp = ['H2 Electrolysis']
c_h2_gen = ['H2 Electrolysis', 'SMR CC']
c_h2_con = ['H2 liquefaction', 'Sabatier', 'Fischer-Tropsch']

In [ ]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

In [ ]:
# stst and exp dataframes
# spatial
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

# temporal
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [ ]:
# Notebook Functions



In [ ]:
#assert 0

## shadow prices

In [ ]:
stst.generators_t.mu_upper.loc[: , stst.generators.carrier == "onwind"]
#stst.links_t.mu_upper.loc[: , stst.links.carrier == "H2 Electrolysis"]

In [ ]:
carriers = ["onwind", "ror", "solar", "solar rooftop", "offwind-dc", "offwind-ac", "hydro", "battery discharger",  "V2G", "PHS" , "urban central gas CHP", "urban central solid biomass CHP CC"]

fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(10, 2*8))

for n, ax, model in zip([stst,exp], axs.reshape(-1), ["STST", "EXP"]):

    for c in carriers:
        if c in n.generators.carrier.unique():
            c_bus = n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]
            gen = n.generators_t.p.loc[: , n.generators.carrier == c]
            mu_up = n.generators_t.mu_upper.loc[:, gen.columns]
            df = pd.concat([pd.DataFrame(mu_up.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        elif c in n.links.carrier.unique():
            c_bus = n.links[n.links.carrier == c]["bus1"].map(n.buses.carrier).unique()[0]
            gen = n.links_t.p1.loc[: , n.links.carrier == c]
            mu_up = n.links_t.mu_upper.loc[:, gen.columns]
            df = pd.concat([pd.DataFrame(mu_up.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        elif c in n.storage_units.carrier.unique():
            c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
            gen = n.storage_units_t.p_dispatch.loc[: , n.storage_units.carrier == c]
            mu_up = n.storage_units_t.mu_upper.loc[:, gen.columns]
            df = pd.concat([pd.DataFrame(mu_up.values.flatten()) , pd.DataFrame(gen.values.flatten())], axis=1)

        df.columns = ["mu_up" , "gen"]
        df = df.sort_values(by="mu_up")
        df["gen_cumsum"] = df["gen"].cumsum()
        df["gen_cumsum_norm"] = df["gen"].cumsum() / df["gen"].sum()
        ax.plot(df["gen_cumsum_norm"], df["mu_up"], color=carrier_colors[c], label= f"{c.replace('urban central', '').replace('solid', '')} ({c_bus})",
                marker=markers[carriers.index(c)], markevery=2920*40)
        ax.set_ylim([-50, 1000])
        ax.set_ylabel("Shadow price of upper p_nom limit [$€/MWh_{el}$]")
        ax.set_xlabel("Fraction of total generation")
        ax.set_facecolor("whitesmoke")
        ax.set_title(f"Electricity generating technologies ({model})", fontsize=16, **font1)
        ax.legend()

fig.tight_layout(pad=3)
plt.show()

# fig.savefig(f"{PLOT_DIR}/el_gen_w_mu_up_within_models.png")

# why is mu_upper negative for storage units?

## regions plot

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(7, 6))
crs = ccrs.EqualEarth()


ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
ax.coastlines(edgecolor='black', linewidth=0.5)
ax.set_facecolor('white')
ax.add_feature(cartopy.feature.OCEAN, color='azure')

df_stst_ons.to_crs(crs.proj4_init).plot(ax=ax,
                                        linewidth=0.3,
                                        facecolor="bisque",
                                        edgecolor = 'grey')

df_stst_off.to_crs(crs.proj4_init).plot(ax=ax,
                                        linewidth=0.3,
                                        facecolor="lavender",
                                        edgecolor = 'grey')


# always select same section
xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
pad = 1 * 1e5  # add a padding around the geometry
ax.set_xlim(xmin-pad, xmax+pad)
ax.set_ylim(ymin-pad, ymax+pad)

plt.show()

fig.savefig(f"C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/01_general/regions.png")